# Pandas for Exploratory Data Analysis II 

Pandas a very useful Python library for data manipulation and exploration. We have so much more to see!

In this lesson, we'll continue exploring Pandas for EDA. Specifically: 

- Identify and handle missing values with Pandas.
- Implement groupby statements for specific segmented analysis.
- Use apply functions to clean data with Pandas.

We'll implicitly review many functions from our first Pandas lesson along the way!

## Remember the AdventureWorks Cycles Dataset?
<img align="right" src="http://lh6.ggpht.com/_XjcDyZkJqHg/TPaaRcaysbI/AAAAAAAAAFo/b1U3q-qbTjY/AdventureWorks%20Logo%5B5%5D.png?imgmax=800">

Here's the Production.Product table [data dictionary](https://www.sqldatadictionary.com/AdventureWorks2014/Production.Product.html), which is a description of the fields (columns) in the table (the .csv file we will import below):<br>
- **ProductID** - Primary key for Product records.
- **Name** - Name of the product.
- **ProductNumber** - Unique product identification number.
- **MakeFlag** - 0 = Product is purchased, 1 = Product is manufactured in-house.
- **FinishedGoodsFlag** - 0 = Product is not a salable item. 1 = Product is salable.
- **Color** - Product color.
- **SafetyStockLevel** - Minimum inventory quantity.
- **ReorderPoint** - Inventory level that triggers a purchase order or work order.
- **StandardCost** - Standard cost of the product.
- **ListPrice** - Selling price.
- **Size** - Product size.
- **SizeUnitMeasureCode** - Unit of measure for the Size column.
- **WeightUnitMeasureCode** - Unit of measure for the Weight column.
- **DaysToManufacture** - Number of days required to manufacture the product.
- **ProductLine** - R = Road, M = Mountain, T = Touring, S = Standard
- **Class** - H = High, M = Medium, L = Low
- **Style** - W = Womens, M = Mens, U = Universal
- **ProductSubcategoryID** - Product is a member of this product subcategory. Foreign key to ProductSubCategory.ProductSubCategoryID.
- **ProductModelID** - Product is a member of this product model. Foreign key to ProductModel.ProductModelID.
- **SellStartDate** - Date the product was available for sale.
- **SellEndDate** - Date the product was no longer available for sale.
- **DiscontinuedDate** - Date the product was discontinued.
- **rowguid** - ROWGUIDCOL number uniquely identifying the record. Used to support a merge replication sample.
- **ModifiedDate** - Date and time the record was last updated.


### Import Pandas

In [1]:
import pandas as pd
import numpy as np # used for linear algebra and random sampling
# used for plotting charts within the notebook (instead of a separate window)
%matplotlib inline

### Read in the dataset

We are using the `read_csv()` method (and the `\t` separator to specify tab-delimited columns).

In [2]:
# read in the dataset

In [3]:
# let's check out the first 3 rows again, for old time's sake

In [4]:
# and the number of rows x cols

### Reset our index (like last time)

Let's bring our `ProductID` column into the index since it's the PK (primary key) of our table and that's where PKs belong as a best practice.

In [5]:
# repalace auto-generated index with the ProductID column

## Handling missing data

Recall missing data is a systemic, challenging problem for data scientists. Imagine conducting a poll, but some of the data gets lost, or you run out of budget and can't complete it! 😮<br><br>

"Handling missing data" itself is a broad topic. We'll focus on two components:

- Using Pandas to identify we have missing data
- Strategies to fill in missing data (known in the business as `imputing`)
- Filling in missing data with Pandas


### Identifying missing data

Before *handling*, we must identify we're missing data at all!

We have a few ways to explore missing data, and they are reminiscient of our Boolean filters.

In [6]:
# True when data isn't missing

In [7]:
# True when data is missing

Now, we may want to see null values in aggregate. We can use `sum()` to sum down a given column

In [8]:
# here is a quick and dirty way to do it

Look! We've found missing values!

How could this missing data be problematic for our analysis?

### Understanding missing data

Finding missing data is the easy part! Determining way to do next is more complicated.

Typically, we are most interested in knowing **why** we are missing data. Once we know what 'type of missingness' we have (the source of missing data), we can proceed effectively.

Let's first quantify how much data we are missing. Here is another implementation of `prod.isnull().sum()`, only wrapped with a `DataFrame` and some labels to make it a little more user-friendly:

In [9]:
# or we can make things pretty as follows

### Filling in missing data

How we fill in data depends largely on why it is missing (types of missingness) and what sampling we have available to us.

We may:

- Delete missing data altogether
- Fill in missing data with:
    - The average of the column
    - The median of the column
    - A predicted amount based on other factors
- Collect more data:
    - Resample the population
    - Followup with the authority providing data that is missing


In our case, let's focus on handling missing values in `Color`. Let's get a count of the unique values in that column. We will need to use the `dropna=False` kwarg, otherwise the `pd.Series.value_counts()` method will not count `NaN` (null) values.

In [10]:
# let's get a value count with the nulls included

Ahoy! We have 248 nulls!

Option 1: Drop the missing values.

In [11]:
# drops rows where any row has a missing value - this does not happen *in place*, so we are not actually dropping

**Important!** `pd.DataFrame.dropna()` and `pd.Series.dropna()` are very versatile! Let's look at the docs (Series is similar):

```python
Signature: pd.DataFrame.dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)
Docstring:
Remove missing values.

See the :ref:`User Guide <missing_data>` for more on which values are
considered missing, and how to work with missing data.

Parameters
----------
axis : {0 or 'index', 1 or 'columns'}, default 0
    Determine if rows or columns which contain missing values are
    removed.

    * 0, or 'index' : Drop rows which contain missing values.
    * 1, or 'columns' : Drop columns which contain missing value.

    .. deprecated:: 0.23.0: Pass tuple or list to drop on multiple
    axes.
how : {'any', 'all'}, default 'any'
    Determine if row or column is removed from DataFrame, when we have
    at least one NA or all NA.

    * 'any' : If any NA values are present, drop that row or column.
    * 'all' : If all values are NA, drop that row or column.
thresh : int, optional
    Require that many non-NA values.
subset : array-like, optional
    Labels along other axis to consider, e.g. if you are dropping rows
    these would be a list of columns to include.
inplace : bool, default False
    If True, do operation inplace and return None.
```

**how**: This tells us if we want to remove a row if _any_ of the columns have a null, or _all_ of the columns have a null.<br>
**subset**: We can input an array here, like `['Color', 'Size', 'Weight']`, and it will only consider nulls in those columns. This is very useful!<br>
**inplace**: This is if you want to mutate (change) the source dataframe. Default is `False`, so it will return a _copy_ of the source dataframe.

To accomplish the same thing, but implement it on our entire dataframe, we can do the following:

In [12]:
# drops all nulls from the Color column, but returns the entire dataframe instead of just the Color column

Option 2: Fill in missing values

Traditionally, we fill missing data with a median, average, or mode (most frequently occurring). For `Color`, let's replace the nulls with the string value `NoColor`.

Let's first look at the way we'd do it with a single column, using the `pd.Series.fillna()` method:

Now let's see how we'd do it to the whole dataframe, using the `pd.DataFrame.fillna()` method. Notice the similar API between the methods with the `value` kwarg. Good congruent design, pandas development team! The full dataframe is returned, and not just a column.

But wait! There's more! We can reference any other data or formulas we want with the imputation (the value we fill the nulls with). This is very handy if you want to impute with the average or median of that column... or even another column altogether! Here is an example where we will the nulls of `Color` with the average value from the `ListPrice` column. This has no practical value in this application, but immense value in other applications.

They're gone! Important points:

- Don't forget to use the `inplace=True` kwarg to mutate the source dataframe (i.e. 'save changes'). 
- It is helpful to not use `inplace=True` initially to ensure your code/logic is correct, prior to making permanent changes.

## Groupby Statements

In Pandas, groupby statements are similar to pivot tables in that they allow us to segment our population to a specific subset.

For example, if we want to know the average number of bottles sold and pack sizes per city, a groupby statement would make this task much more straightforward.


To think how a groupby statement works, think about it like this:

- **Split:** Separate our DataFrame by a specific attribute, for example, group by `Color`
- **Combine:** Put our DataFrame back together and return some _aggregated_ metric, such as the `sum`, `count`, or `max`.

![](http://i.imgur.com/yjNkiwL.png)

Let's try it out!

Let's group by `Color`, and get a count of products for each color.

In [13]:
# group by Color, giving the number of products of each color

What do we notice about this output? Are all columns the same? Why or why not?

We can see that the `.count()` method excludes nulls, and there is no way to change this with the current implementation:
```python
Signature: .count()
Docstring: Compute count of group, excluding missing values 
File:      ~/miniconda3/envs/ga/lib/python3.7/site-packages/pandas/core/groupby/groupby.py
Type:      method
```

As a best practice, you should either:
- fill in nulls prior to your .count(), or
- use the PK (primary key) of the table, which is guaranteed non-null

In [14]:
# here we can use 'x' as a dummy placeholder for nulls, simply to get consistent counts for all columns

Let's find out the most expensive price for an item, by `Color`:

We can also do multi-level groupbys. This is referred to as a `Multiindex` dataframe. Here, we can see the following fields in a nested group by, with a count of Name (with nulls filled!); effectively giving us a count of the number of products for every unique Class/Style combination:

- Class - H = High, M = Medium, L = Low
- Style - W = Womens, M = Mens, U = Universal

We can also use the `.agg()` method with multiple arguments, to simulate a `.describe()` method like we used before:

## Apply functions for column operations

Apply functions allow us to perform a complex operation across an entire columns highly efficiently.

For example, let's say we want to change our colors from a word, to just a single letter. How would we do that?

The first step is writing a function, with the argument being the value we would receive from each cell in the column. This function will mutate the input, and return the result. This result will then be _applied_ to the source dataframe (if desired).

In [15]:
# get a list of all unique colors

In [16]:
# create a function to convert colors to letters

Now we can _apply_ this function to our `pd.Series` object, returning the result (which we can use to overwrite the source, if we choose).

The `pd.DataFrame.apply` implementation is similar, however it effectively 'scrolls through' the columns and passes each one sequentially to your function:

```python
Objects passed to the function are Series objects whose index is
either the DataFrame's index (``axis=0``) or the DataFrame's columns
(``axis=1``).
```

It should only be used when you wish to apply the same function to all columns (or rows) of your `pd.DataFrame` object.

We can also use `pd.Series.apply()` with a **labmda expression**. This is an undeclared function and is commonly used for simple functions within the `.apply()` method. Let's use it to add $100 to our `ListPrice` column. Hey, baby needs new shoes!

In [17]:
# prior to apply method

In [18]:
# and now with 100 more dollars!

Boom! Maybe financing that new boat wasn't such a bad idea after all!

**Your turn:** Identify one other column where we may want to write a new apply function, or use the one we just created for the purposes of cleaning up our dataset.

In [19]:
# identify a column to mutate (change)


In [20]:
# write a function to mutate that column (or columns) note: if using a lambda function, you can leave this blank


In [21]:
# apply that function across the whole column


## Wrap up

We've covered even more useful information! Here are the key takeaways:

- **Missing data** comes in many shapes and sizes. Before deciding how to handle it, we identify it exists. We then derive how the missingness is affecting our dataset, and make a determination about how to fill in values.

```python
# pro tip for identifying missing data
df.isnull().sum()
```

- **Grouby** statements are particularly useful for a subsection-of-interest analysis. Specifically, zooming in on one condition, and determining relevant statstics.

```python
# group by 
df.groupby('column').agg['count', 'mean', 'max', 'min']
```

- **Apply functions** help us clean values across an entire DataFrame column. They are *like* a for loop for cleaning, but many times more efficient. They follow a common pattern:
1. Write a function that works on a single value
2. Test that function on a single value
3. Apply that function to a whole column

(The most confusing part of apply functions is that we write them with *a single value* in mind, and then apply them to many single values at once.)